1. 图片读取 dataload

In [ ]:
!pip uninstall torch

Looking in indexes: https://download.pytorch.org/whl/cu121


In [10]:
#!conda install pytorch torchvision torchaudio pytorch-cuda==12.2 -c pytorch -c nvidia
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

^C


In [11]:
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision import models
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import ImageFolder
import torch.nn as nn
import torch.optim as optim
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
dataset = ImageFolder(root='./images/', transform=transform)
train_size = int(0.7 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=256, shuffle=False)



KeyboardInterrupt



In [ ]:
print(torch.__version__)

In [ ]:
!pip install tensorflow

In [ ]:
torch.cuda.empty_cache()

In [ ]:
import time
from tqdm import tqdm

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model_save_path = 'mobilenet/resnet50.pth'
resnet50 = models.resnet50(pretrained=True)
num_ftrs = resnet50.fc.in_features
resnet50.fc = nn.Linear(num_ftrs, 12)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(resnet50.parameters(), lr=0.001)
num_epochs = 10  # 举例，可以根据需要调整
resnet50 = resnet50.to('cuda:0')

for epoch in range(num_epochs):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    resnet50.train()  # 设置模型为训练模式
    running_loss = 0.0
    print(device)
    train_correct = 0

    train_total = 0
    for i, (inputs, labels) in enumerate(tqdm(train_loader)):
        inputs = inputs.to(device)
        labels = labels.to(device)

        train_start_time = time.time()

        optimizer.zero_grad()  # 清零梯度
        outputs = resnet50(inputs)  # 前向传播
        loss = criterion(outputs, labels)  # 计算损失
        loss.backward()  # 反向传播
        optimizer.step()  # 更新权重
        train_end_time = time.time()
        print(f'Training completed in {train_end_time - train_start_time} seconds')
        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        train_total += labels.size(0)
        train_correct += (predicted == labels).sum().item()

    train_loss = running_loss / len(train_loader)
    train_acc = 100 * train_correct / train_total
    torch.save(resnet50.state_dict(), model_save_path)
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader)}')
    print(f'Epoch [{epoch+1}/{num_epochs}], Acc: {100 * train_correct / train_total}')

    # 验证模型
    val_running_loss = 0.0
    val_correct = 0
    val_total = 0
    with torch.no_grad():  # 在验证过程中不计算梯度
        resnet50.eval()  # 设置模型为评估模式
        for j, (inputs, labels) in enumerate(tqdm(val_loader)):
            train_start_time = time.time()
            outputs = resnet50(inputs)
            loss = criterion(outputs, labels)
            val_running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()
            train_end_time = time.time()
            print(f'val completed in {train_end_time - train_start_time} seconds')

    val_loss = val_running_loss / len(val_loader)
    val_acc = 100 * val_correct / val_total

    print(f'Epoch {epoch+1}, Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%')



In [ ]:
import numpy as np
from tensorflow.keras.optimizers import Adam
import cv2
from tensorflow.keras.preprocessing.image import img_to_array
from sklearn.model_selection import train_test_split
from tensorflow.python.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.applications.resnet50 import ResNet50
import os
from tensorflow.python.keras.utils import np_utils
from tensorflow.python.keras.layers import Dense
from tensorflow.python.keras.models import Sequential
import keras.utils
from keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.models import Model

In [ ]:
import cv2
from tensorflow.keras.preprocessing.image import img_to_array
import numpy as np

imageList = []
labelList = []
norm_size = 112
datapath = './jpg灰度'
path=os.path.join(datapath)
image_names=os.listdir(path)
image_names

In [ ]:
file = open("image_names.txt",'w')
for fp in image_names:
  file.write(str(fp))
  file.write('\n')
file.close()

In [ ]:
count = 0
labelList = []
imageList = []
for image_name in image_names[:1000]:
  image_full_path = os.path.join(path,image_name)
  label = image_name.split('_')
  label = int(label[0])
  labelList.append(label)
  image = cv2.imdecode(np.fromfile(image_full_path, dtype=np.uint8), cv2.IMREAD_COLOR)
  image = cv2.resize(image,(norm_size,norm_size),interpolation=cv2.INTER_LANCZOS4)
  # if image.shape[2]>3:
  #   image=image[:,:,:3]
  #   print(image.shape)
  image = img_to_array(image)
  print('count:',count)
  imageList.append(image)
  count = count+1

imageList = np.array(imageList)/255.0

In [ ]:
import pickle
from sklearn import preprocessing
print(labelList)
lb = preprocessing.LabelBinarizer()
labelList = lb.fit_transform(labelList)
print(labelList)
print(lb.classes_)
f = open('label_list.pickle',"wb")
f.write(pickle.dumps(lb))
f.close()

In [ ]:
imageList.shape

In [ ]:
labelList.shape

In [ ]:
trainX,valX,trainY,valY = train_test_split(imageList,labelList,test_size=0.2, random_state=42)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rotation_range=20,
                  horizontal_flip=True)
train_data = train_datagen.flow(trainX,trainY,batch_size=16,shuffle=True)

val_datagen = ImageDataGenerator()
val_generator=val_datagen.flow(valX,valY,batch_size=16,shuffle=True)

checkpointer = ModelCheckpoint(filepath='resnet_best.hdf5',monitor='val_accuracy',verbose=1,save_best_only=True,mode='max')

reduce = ReduceLROnPlateau(monitor='val_accuracy',patience=10,verbose=1,factor=0.5,min_lr=1e-6)

In [ ]:
batch_size = 16
net=ResNet50(include_top=False,weights='imagenet',input_tensor=None,input_shape=(112,112,3))
x=net.output
print(x.shape)
x = Flatten()(x)
print(x.shape)
x = Dropout(0.2)(x)
output_layer = Dense(12,activation='softmax',name='softmax')(x)
net_classfier =Model(inputs=net.input, outputs=output_layer)
# for layer in net_final.layers[:freeze_layers]:
#     layer.trainable = False
# for layer in net_final.layers[freeze_layers:]:
#     layer.trainable = True

net_classfier.compile(optimizer=Adam(lr=1e-3),loss='categorical_crossentropy', metrics=['accuracy'])

print(net_classfier.summary())

net_classfier.fit(train_data,steps_per_epoch=trainX.shape[0]/batch_size,
         validation_data=val_generator,
         epochs=100,
         validation_steps=valX.shape[0] / batch_size,
         callbacks=[checkpointer, reduce])


In [ ]:
net_classfier.save('resnet50.h5')